In [1]:
import nglview as nv
from Bio.PDB.MMCIFParser import MMCIFParser
from Bio.PDB.PDBList import PDBList
from Bio.PDB.Polypeptide import three_to_one
from Bio import Align
from Bio.Align import substitution_matrices
import Bio
import sys

def get_sequence(residues):
    return ''.join(three_to_one(r.get_resname()) for r in residues)

def get_residues(structure, chain):
    return [r for r in structure[0][chain].get_residues() if r.id[0] == ' ']

In [10]:
def superpose(pdb_ids, viewer, chain_ids = [None, None]):

    pdbl = PDBList()
    parser = MMCIFParser()
    
    structures = [None, None]
    chains = [[],[]]
    for i in [0,1]:
        pdb_id = pdb_ids[i]
        fn = pdbl.retrieve_pdb_file(pdb_id, file_format="mmCif", pdir=".")        
        s = parser.get_structure(pdb_id, fn)        
        structures[i] = s
        chains[i] = [x.id for x in s.get_chains() if chain_ids[0] is None or x.id == chain_ids[i] ]
        print(chains[i])

    aligner = Align.PairwiseAligner()
    aligner.substitution_matrix = substitution_matrices.load("BLOSUM62")
    aligner.open_gap_score = -11
    aligner.end_open_gap_score = -11
    aligner.extend_gap_score = -1
    aligner.end_extend_gap_score = -1

    super_imposer = Bio.PDB.Superimposer()

    min_rms = sys.float_info.max
    min_atoms = None
    print(chains)
    for i in range(len(chains[0])):
        for j in range(len(chains[1])):
            res = [get_residues(structures[0], chains[0][i]), get_residues(structures[1], chains[1][j])]
            seq = [get_sequence(res[0]), get_sequence(res[1])]      
       
            alns = aligner.align(seq[0], seq[1])

            mapping = [[],[]]    
            for range1, range2 in zip(alns[0].aligned[0], alns[0].aligned[1]):
                mapping[0] += list(range(range1[0], range1[1]))
                mapping[1] += list(range(range2[0], range2[1]))
                
            atoms = [[],[]]
            for ix in range(2):        
                for ix_ix in mapping[ix]:
                    atoms[ix].append([a for a in res[ix][ix_ix].get_atoms() if a.get_name() == 'CA'][0])
            
            super_imposer.set_atoms(atoms[0], atoms[1])
            super_imposer.apply(structures[1].get_atoms())
            rms = super_imposer.rms
            print(f"RMSD of chains {chains[0][i]} and {chains[1][j]}: {rms}")
            if rms < min_rms:
                min_rms = rms
                min_atoms = [atoms[0], atoms[1]]
            
    super_imposer.set_atoms(min_atoms[0], min_atoms[1])
    super_imposer.apply(structures[1].get_atoms())
    
    c1 = viewer.add_component(nv.BiopythonStructure(structures[0][0]))
    c2 = viewer.add_component(nv.BiopythonStructure(structures[1][0]))

    c1.clear_representations()
    c1.add_representation('cartoon', color="blue")
    c2.clear_representations()
    c2.add_representation('cartoon', color="red")
    
        


In [13]:

v = nv.NGLWidget()
    
# superpose(pdb_ids=['7E2X', '6A94'], viewer=v)
# superpose(pdb_ids=['4n58', '4n59'], viewer=v)
superpose(pdb_ids=['4n58', '4n59'], viewer=v, chain_ids=['B', 'B'])
v

Structure exists: '.\4n58.cif' 


c:\Python39\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4173.
  warnings.warn(
c:\Python39\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4356.
  warnings.warn(
c:\Python39\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4562.
  warnings.warn(
c:\Python39\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4792.
  warnings.warn(


['B']
Structure exists: '.\4n59.cif' 


c:\Python39\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4091.
  warnings.warn(
c:\Python39\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4106.
  warnings.warn(
c:\Python39\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4125.
  warnings.warn(
c:\Python39\lib\site-packages\Bio\PDB\StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 4151.
  warnings.warn(


['B']
[['B'], ['B']]
RMSD of chains B and B: 10.563220557025248


NGLWidget()